In [1]:
from lib.DataHandler import DataAcquisitionHandler
from abc import abstractmethod
import pickle
import torch
import scipy.signal as signal
import numpy as np

pygame 2.5.1 (SDL 2.28.2, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
filename = "C:/Users/c25th/code/P300_BCI_Speller/data/handlers/handler_2023-11-23.pkl"
with open(filename, 'rb') as f:
            handler = pickle.load(f)
print(handler.get_data())

{'box_data': [{'metadata': {'start_time': 1700766442.0747888, 'length': 227.86162543296814, 'trials': [{'timestamp': (1.7131330966949463, 2.7382893562316895), 'label': True}, {'timestamp': (5.152762174606323, 6.183048486709595), 'label': True}, {'timestamp': (8.704437017440796, 9.739725589752197), 'label': True}, {'timestamp': (11.353266716003418, 12.383774518966675), 'label': True}, {'timestamp': (14.000005960464478, 15.000250816345215), 'label': False}, {'timestamp': (16.916667699813843, 17.916971683502197), 'label': False}, {'timestamp': (19.530187129974365, 20.53031849861145), 'label': False}, {'timestamp': (22.645355939865112, 23.665996074676514), 'label': True}, {'timestamp': (25.980265617370605, 27.0091290473938), 'label': True}, {'timestamp': (28.822618007659912, 29.822794914245605), 'label': False}, {'timestamp': (31.639228582382202, 32.63948583602905), 'label': False}, {'timestamp': (34.758564472198486, 35.794076919555664), 'label': True}, {'timestamp': (38.30941820144653, 39

In [5]:
class DataObject:


    def __init__(self, data_dict):

        self.keyboard_sessions = []

        if 'keyboard_data' in data_dict.keys():
            keyboard_session_list = data_dict['keyboard_data']

            for session_dict in keyboard_session_list:
                self.keyboard_sessions.append(SessionData(session_dict=session_dict, type='keyboard'))

        self.box_sessions = []

        if 'box_data' in data_dict.keys():
            box_session_list = data_dict['box_data']

            for session_dict in box_session_list:
                self.box_sessions.append(SessionData(session_dict=session_dict, type='box'))


    def get_data(self, decorator=None, type='box'):
        if decorator is None:
            keyboard_data = self.keyboard_sessions
            box_data = self.box_sessions
        else:
            keyboard_data, box_data = decorator.visit_data_object(object=self)
        return (keyboard_data, box_data)
    
    def accept(self, visitor):
        visitor.visit_data_object(object=self)




class SessionData:
    
    def __init__(self, session_dict, type):
        metadata = session_dict['metadata']
        self.data = session_dict['data']
        self.start_time = metadata['start_time']
        self.length = metadata['length']
        self.flash_time_range = metadata['flash_time_range']
        self.sample_time = metadata['sample_time']
        self.description = metadata['description']
        self.type = type

        self.trials = []
        for trial in metadata['trials']:
            if self.type == 'keyboard':
                self.trials.append(KeyboardTrialData(trial_dict=trial, parent_session=self))
            elif self.type == 'box':
                self.trials.append(BoxTrialData(trial_dict=trial, parent_session=self))
            else:
                raise ValueError('Session type must be either keyboard or box')

    def get_data(self, decorator=None):
        if decorator is None:
            trials = self.trials
        else:
            trials = decorator.visit_session_data(session=self)
        return trials
    
    def accept(self, visitor):
        visitor.visit_session_data(session=self)



class TrialData:

    def __init__(self, trial_dict, parent_session):
        self.timestamp = trial_dict['timestamp']
        self.label = trial_dict['label']
        self.parent_session = parent_session

    def get_data(self, decorator=None):
        if decorator is None:
            data = self.data
        else:
            data = decorator.visit_trial_data(trial=self)
        return data
    
    def accept(self, visitor):
        visitor.visit_trial_data(trial=self)
    


class BoxTrialData(TrialData):

    def __init__(self, trial_dict, parent_session):
        super().__init__(trial_dict, parent_session)



class KeyboardTrialData(TrialData):

    def __init__(self, trial_dict, parent_session):
        super().__init__(trial_dict, parent_session)
        self.pattern = trial_dict['pattern']
        self.letter = trial_dict['letter']

In [6]:
class DataDecorator(object):

    def visit_data_object(object):
        raise NotImplementedError
    
    def visit_session_data(session):
        raise NotImplementedError
    
    def visit_trial_data(trial, raw_data):
        raise NotImplementedError


##########################################################################################


class MakeWindowsDataDecorator(DataDecorator):
    """
    Returns a list of tuples ( data_window, label )
    """

    def __init__(self):
        pass


    def visit_data_object(self, object):
        """
        Returns a tuple of lists of tuples ( [( data_windows, label ) ...], [( data_windows, label ) ...] )
        This first is the keyboard data, the second is the box data
        """
        keyboard_sessions = object.keyboard_sessions
        box_sessions = object.box_sessions

        keyboard_data = []
        for session in keyboard_sessions:
            keyboard_data.extend(self.visit_session_data(session=session))

        box_data = []
        for session in box_sessions:
            box_data.extend(self.visit_session_data(session=session))

        return ( keyboard_data, box_data)
    

    def visit_session_data(self, session):
        """
        Returns a list of tuples ( data_windows, label )
        """
        data = []
        for trial in session.trials:
            data.append(self.visit_trial_data(trial=trial))
        return data
    

    def visit_trial_data(self, trial):
        """
        Returns a tuple ( data_windows, label )
        """
        start = trial.timestamp[0]
        end = start + trial.parent_session.sample_time

        # Make it an index for the data rather than a time
        data_len = len(trial.parent_session.data[0])
        start = (int) ( (start / trial.parent_session.length) * data_len )
        end = (int) ( (end / trial.parent_session.length) * data_len )

        window = self.transform_window(trial.parent_session.data[:, start:end])
        label = self.transform_label(trial.label)

        return ( window, label )
    
    def transform_window(window):
        return window
    
    def transform_label(label):
        return label
    

class MakeTensorWindowsDataDecorator(MakeWindowsDataDecorator):
    """
    Returns a list of tuples ( data_window, label ) for each window in the format of a pytorch tensor
    """

    def transform_label(self, label):
        if not label:
            label = 0
        elif label:
            label = 1
        else:
            raise ValueError('MakeTensorWindowsDataVisitor object/ transform_label method: Label must be a boolean')
        
        return torch.tensor(label)
    
    def transform_window(self, window):
        return torch.from_numpy(window).float()

In [26]:
class DataVisitor(object):

    def visit_data_object(self, object):
        raise NotImplementedError
    
    def visit_session_data(self, session):
        raise NotImplementedError

    def visit_trial_data(self, trial):
        raise NotImplementedError
    

##########################################################################################


class FilterVisitor(DataVisitor):
    """
    Modifies the data inside the DataObject to be bandpass filtered
    """

    SAMPLE_RATE = 250
    ORDER = 3

    def __init__():
        raise NotImplementedError

    def visit_data_object(self, object):
        for session in object.keyboard_sessions:
            self.visit_session_data(session=session)

        for session in object.box_sessions:
            self.visit_session_data(session=session)
    
    def visit_session_data(self, session):
        data = []
        for i, channel in enumerate(session.data):
            # Only the channels with EM data will be filtered
            if i>1 and i<9:
                data.append(self.filter(channel))
            else:
                data.append(channel)

        session.data = np.array(data)

    def visit_trial_data(self, trial):
        pass

    def filter(self, data):
        raise NotImplementedError
    

class BandpassFilterVisitor(FilterVisitor):

    def __init__(self, low, high):
        self.low = low
        self.high = high

    def filter(self, data):
        """Bandpass filter using scipy.signal.butter"""
        b, a = signal.butter(3, [self.low, self.high], btype='bandpass', fs=self.SAMPLE_RATE)
        return signal.lfilter(b, a, data)
    

class BandstopFilterVisitor(FilterVisitor):
    
    def __init__(self, low, high):
        self.low = low
        self.high = high
    
    def filter(self, data):
        """Bandstop filter using scipy.signal.butter"""
        b, a = signal.butter(3, [self.low, self.high], btype='bandstop', fs=self.SAMPLE_RATE)
        return signal.lfilter(b, a, data)
    

class CommonAveragingVisitor(DataVisitor):
    
    def __init__(self):
        pass

    def visit_data_object(self, object):
        for session in object.keyboard_sessions:
            self.visit_session_data(session=session)

        for session in object.box_sessions:
            self.visit_session_data(session=session)

    def visit_session_data(self, session):
        data = []
        for i, channel in enumerate(session.data):
            # Only the channels with EM data will be filtered
            if i>1 and i<9:
                data.append(self.common_average(channel))
            else:
                data.append(channel)

    def common_average(self, channel):
        avg_signal = np.mean(channel, axis=0)
        avg_final = channel - avg_signal
        norm_data = (avg_final - np.min(avg_final)) / (np.max(avg_final) - np.min(avg_final))
        final_data = np.sqrt(norm_data)
        return final_data
    

class SmoothDataVisitor(DataVisitor):

    SAMPLE_RATE = 250

    def __init__(self, n=15, low=0.1, high=30):
        self.n = n
        self.b = [1.0/n] * n
        _, self.a = signal.butter(3, [low, high], btype='bandpass', fs=self.SAMPLE_RATE)

    def visit_data_object(self, object):
        for session in object.keyboard_sessions:
            self.visit_session_data(session=session)

        for session in object.box_sessions:
            self.visit_session_data(session=session)

    def visit_session_data(self, session):
        data = []
        for i, channel in enumerate(session.data):
            # Only the channels with EM data will be filtered
            if i>1 and i<9:
                data.append(self.smooth(channel))
            else:
                data.append(channel)

    def smooth(self, channel):
        return signal.lfilter(self.b, self.a, channel)

In [27]:
# Data filter visitor test

data = DataObject(handler.get_data())

data.accept(BandpassFilterVisitor(low=0.1, high=10))
data.accept(BandstopFilterVisitor(low=49, high=51))
data.accept(CommonAveragingVisitor())
data.accept(SmoothDataVisitor())

formatted_data = data.get_data(decorator=MakeTensorWindowsDataDecorator())

print(len(formatted_data[1]))
print(len(formatted_data[1][0]))
print(len(formatted_data[1][0][0]))
print(len(formatted_data[1][0][0][0]))
print(len(formatted_data[1][2][0][0]))

438
2
24
248
248


C:\Users\c25th\AppData\Local\Temp\ipykernel_3252\3443658733.py:100: RuntimeWarning: invalid value encountered in divide
  norm_data = (avg_final - np.min(avg_final)) / (np.max(avg_final) - np.min(avg_final))


In [19]:
# Data Decorator Test

# data = DataObject(handler.get_data())

# formatted_data = data.get_data(decorator=MakeTensorWindowsDataDecorator())

# print(len(formatted_data[1]))
# print(len(formatted_data[1][0]))
# print(len(formatted_data[1][0][0]))
# print(len(formatted_data[1][0][0][0]))
# print(len(formatted_data[1][2][0][0]))

83
2
24
248
248


In [20]:
print(formatted_data[1][0][1])

tensor(0)
